<a href="https://colab.research.google.com/github/eleshuk/machine_learning/blob/main/eleshuk_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create a script to train a regression model to predict the response variable "quality" given the available explanatory variables for the Wine Quality data set (https://archive.ics.uci.edu/dataset/186/wine+quality) that contains 4898 examples.

In [70]:
from google.colab import drive
import pandas as pd
import zipfile
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Mount drive to access zip folder in google drive
drive.mount('/content/drive/')

In [10]:
file_path = "/content/drive/My Drive/wine+quality.zip"
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
print(file_list)

['winequality-red.csv', 'winequality-white.csv', 'winequality.names']


In [16]:
zip_path = file_path
extract_to = "wine_quality_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [78]:
# Look at white wine first, that's the dataset that has 4898 examples (later combine red and white or does it make more sense to keep them separate sine maybe indicators of quality are different?)
df = pd.read_csv("wine_quality_data/winequality-white.csv", sep = ';')
# df.head()

In [25]:
# Read metadata file
# with open("wine_quality_data/winequality.names", "r") as file:
#     print(file.read())

predict the response variable "quality" given the available explanatory variables for the Wine Quality data set

In [86]:
# PyTorch expects numeric data in float32 (for continuous data) - we have a pandas df rn bc of the way that we read in the data

# Define features (X) and target (y)
features = df.drop(columns=["quality"])  # Drop target variable
y_df = df["quality"]  # Target variable

# Convert features to a PyTorch tensor
X = torch.tensor(features.values, dtype=torch.float32)
y = torch.tensor(y_df.astype(np.float32))
y = y.view(y.shape[0], 1)

tensor([[6.],
        [6.],
        [6.],
        ...,
        [6.],
        [7.],
        [6.]])

In [88]:
# Samples - samples within the dataset
# Features - 11

# Define number of samples and features
n_samples, n_features = X.shape
# input size - number of features (11)
# output size - quality (1)

4898 11


In [91]:
# Linear model f = wx + b
input_size = n_features
output_size = 1 # (quality)
model = nn.Linear(input_size, output_size)
model

Linear(in_features=11, out_features=1, bias=True)

In [95]:
# Loss and optimizer
learning_rate = 0.01

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [98]:
# y_predicted = model(X)

tensor([[21.9072],
        [23.9398],
        [11.1015],
        ...,
        [14.7738],
        [17.7923],
        [14.5116]], grad_fn=<AddmmBackward0>)

In [81]:
# 3) Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass and loss
    y_predicted = model(X)
    print(y_predicted)
    loss = criterion(y_predicted, y)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([4898])) that is different to the input size (torch.Size([4898, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
  

KeyboardInterrupt: 

In [27]:
# Start by standardizing the data - benefits = not all numeric explanatory variables are of the same scale, so standardizing prevents the model from giving importance to a variable of a larger scale



In [31]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)